# Part-1 : Section A
Training a multi-exit ElasticBERT model on SST-2 dataset

In [5]:
# The code closely follows the original ElasticBERT repository
# Feature to train models with a given exit configuration is added
!git clone https://github.com/MLiONS/MutiExitDNNs.git


%cd /content/MutiExitDNNs/ElasticBERT
!pip install -r requirements.txt

Cloning into 'MutiExitDNNs'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 179 (delta 58), reused 0 (delta 0), pack-reused 80
Receiving objects: 100% (179/179), 2.27 MiB | 10.43 MiB/s, done.
Resolving deltas: 100% (94/94), done.
[Errno 2] No such file or directory: '/content/MutiExitDNNs/ElasticBERT'
/content/ElasticBERT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#All the hyper-parameters/ location to training dataset are set in
#MultiExitDNNs -> finetune-dynamic -> finetune_elue_entropy.sh file

#1)Set the correct location to SST-2 dataset
#All models are trained on SST-2 "train" split and evaluated on "dev" split
#"train.tsv" and "dev.tsv" are expected to be in ELUE_DIR/TASK_NAME
#You can set both ELUE_DIR and TASK_NAME in finetune_elue_entropy.sh
#Or change the dataset directory using "data_dir" option

#2)Please change the "num_output_layers" option as per the desired exit-configuration

#3)Model checkpoints will be saved at "output_dir" and
#logs will be available at "log_dir"

!bash /content/MutiExitDNNs/ElasticBERT/finetune-dynamic/finetune_elue_entropy.sh

bash: /content/MutiExitDNNs/ElasticBERT/finetune-dynamic/finetune_elue_entropy.sh: No such file or directory


# Part-1 : Section B
Generating the prediction matrix

In [ ]:
#Evaluation on other datasets-IMDb or Yelp

In [ ]:
from transformers import BertTokenizer as ElasticBertTokenizer

#Set the current directory location inside "finetune-dynamic" folder
%cd /content/MutiExitDNNs/ElasticBERT/finetune-dynamic

from models.configuration_elasticbert import ElasticBertConfig
from models.modeling_elasticbert_entropy import ElasticBertForSequenceClassification

/content/MutiExitDNNs/ElasticBERT/finetune-dynamic


In [ ]:
#Set location to the best performing model
#Model checkpoints are saved at "output_dir" from Part-1: Section A

checkpoint = '/content/MutiExitDNNs/ElasticBERT/ckpts/elue/entropy/SST-2/checkpoint-50'

In [ ]:
config = ElasticBertConfig.from_pretrained(checkpoint)
tokenizer = ElasticBertTokenizer.from_pretrained(checkpoint)
model = ElasticBertForSequenceClassification.from_pretrained(checkpoint)
#model

In [ ]:
def get_args(arg_vec):
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument(
        "--num_hidden_layers",
        default=None,
        type=int,
        required=True,
        help='The number of layers to import.',
    )
    parser.add_argument(
        "--num_output_layers",
        nargs = 12,
        default=None,
        type=int,
        required=True,
        help='The number of layers to output.',
    )
    parser.add_argument(
        "--data_dir",
        default=None,
        type=str,
        required=True,
        help="The input data dir. Should contain the .tsv files (or other data files) for the task.",
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        required=True,
        help="Path to pre-trained model or shortcut name.",
    )
    parser.add_argument(
        "--task_name",
        default=None,
        type=str,
        required=True,
        help="The name of the task to train selected in the list.",
    )
    parser.add_argument(
        "--output_dir",
        default=None,
        type=str,
        required=True,
        help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument(
        "--log_dir",
        default=None,
        type=str,
        required=True,
        help="The output directory where the logs will be written.",
    )
    parser.add_argument(
        "--spec_eval",
        default=None,
        type=str,
        required=False,
        help="'Set as train or test based on specific split on which to evaluate'"
    )
    parser.add_argument(
        "--patience",
        default='0',
        type=str,
        required=False,
    )
    parser.add_argument(
        "--regression_threshold",
        default=0,
        type=float,
        required=False,
    )
    parser.add_argument(
        "--early_exit_entropy",
        default='0.1',
        type=str,
        required=False,
    )
    # Other parameters
    parser.add_argument(
        "--load",
        default=None,
        type=str,
        help="The path of ckpts used to continue training."
    )
    parser.add_argument(
        "--config_name",
        default="",
        type=str,
        help="Pretrained config name or path if not the same as model_name",
    )
    parser.add_argument(
        "--tokenizer_name",
        default="",
        type=str,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--cache_dir",
        default="",
        type=str,
        help="Where do you want to store the pre-trained models downloaded from huggingface.co",
    )
    parser.add_argument(
        "--max_seq_length",
        default=128,
        type=int,
        help="The maximum total input sequence length after tokenization. Sequences longer "
             "than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument("--debug", action="store_true", help="Whether to use debug mode.")
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument(
        "--evaluate_during_training",
        action="store_true",
        help="Run evaluation during training at each logging step.",
    )
    parser.add_argument(
        "--do_lower_case",
        action="store_true",
        help="Set this flag if you are using an uncased model.",
    )
    parser.add_argument(
        "--per_gpu_train_batch_size",
        default=8,
        type=int,
        help="Batch size per GPU/CPU for training.",
    )
    parser.add_argument(
        "--per_gpu_eval_batch_size",
        default=1,
        type=int,
        help="Batch size per GPU/CPU for evaluation.",
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument(
        "--learning_rate",
        default=5e-5,
        type=float,
        help="The initial learning rate for Adam.",
    )
    parser.add_argument("--weight_decay", default=0.01, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument(
        "--num_train_epochs",
        default=3.0,
        type=float,
        help="Total number of training epochs to perform.",
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")
    parser.add_argument("--warmup_rate", default=0, type=float, help="Linear warmup over warmup_rate.")

    parser.add_argument("--logging_steps", type=int, default=500, help="Log every X updates steps.")
    parser.add_argument(
        "--save_steps",
        type=int,
        default=500,
        help="Save checkpoint every X updates steps.",
    )
    parser.add_argument(
        "--eval_all_checkpoints",
        action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
    )
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument(
        "--overwrite_output_dir",
        action="store_true",
        help="Overwrite the content of the output directory",
    )
    parser.add_argument(
        "--overwrite_cache",
        action="store_true",
        help="Overwrite the cached training and evaluation sets",
    )
    parser.add_argument(
        "--not_save_model",
        action="store_true",
        help="Do not save model checkpoints"
    )
    parser.add_argument("--seed", type=int, default=6, help="random seed for initialization")

    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    parser.add_argument(
        "--fp16_opt_level",
        type=str,
        default="O1",
        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
             "See details at https://nvidia.github.io/apex/amp.html",
    )
    parser.add_argument(
        "--local_rank",
        type=int,
        default=-1,
        help="For distributed training: local_rank",
    )
    parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")
    args = parser.parse_args(arg_vec)

    return args

In [ ]:
from load_data import (
    load_and_cache_examples_glue,
    load_and_cache_examples_elue,
)

def evaluate_elue_entropy(args, model, tokenizer, prefix="", eval_highway=False, entropy=0.):
    model.elasticbert.set_early_exit_entropy(entropy)
    model.elasticbert.set_eval_state(eval_highway)
    model.elasticbert.reset_stats()

    eval_task = args.task_name.lower()
    eval_output_dir = args.output_dir

    num_op_layers = args.num_output_layers

    results = {}
    results_all = []
    exit_layer = []
    for i in range(sum(num_op_layers)):
        results_all.append({})

    if args.spec_eval:
      eval_dataset = load_and_cache_examples_elue(args, eval_task, tokenizer, data_type=args.spec_eval)
    else:
      eval_dataset = load_and_cache_examples_elue(args, eval_task, tokenizer, data_type='dev')

    if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # multi-gpu eval
    if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    preds_all = []
    pred_tuple = []
    for i in range(sum(num_op_layers)):
        preds_all.append(None)
        pred_tuple.append(None)
    out_label_ids = None

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[-1],
            }
            inputs["token_type_ids"] = batch[2]
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if out_label_ids is None:
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
        if not eval_highway:
            for i, pred in enumerate(preds_all):
                if pred is None:
                    preds_all[i] = logits[i].detach().cpu().numpy()
                else:
                    preds_all[i] = np.append(pred, logits[i].detach().cpu().numpy(), axis=0)
        else:
            if preds is None:
                preds = logits.detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args.output_mode == "classification":
        if not eval_highway:
            for i, pred in enumerate(preds_all):
                preds_all[i] = np.argmax(pred, axis = 1)
                pred_tuple[i] = pred
        else:
            preds = np.argmax(preds, axis = 1)
            pred_tuple[i] = pred

    elif args.output_mode == "regression":
        if not eval_highway:
            for i, pred in enumerate(preds_all):
                preds_all[i] = np.squeeze(pred)
        else:
            preds = np.squeeze(preds)

    if not eval_highway:
        for i, pred in enumerate(preds_all):
            if eval_task == 'yelp':
              eval_task = 'imdb'
            result = elue_compute_metrics(eval_task, pred, out_label_ids)
            results_all[i].update(result)

    else:
        result = elue_compute_metrics(eval_task, preds, out_label_ids)
        results.update(result)

        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            print("  %s = %s" % (key, str(result[key])))

        exiting_layer_every_ins = model.elasticbert.exiting_layer_every_ins
        exit_layer.append(exiting_layer_every_ins)

    if eval_highway:
        speed_up = model.elasticbert.log_stats()
        return results, speed_up, exit_layer

    if args.spec_eval:
      return results_all, preds_all, pred_tuple, out_label_ids

    return results_all, preds_all, pred_tuple , output_label_ids
    #return results_all, preds_all, out_label_ids


In [ ]:
ELUE_DIR='/content/drive/MyDrive/elue_data'
TASK_NAME='SST-2'

arg_vec= ['--model_name_or_path', 'fnlp/elasticbert-base',
  '--task_name', 'SST-2', \
  '--do_train', \
  '--do_lower_case', \
  '--data_dir', "/content/drive/MyDrive/elue_data/STS-B", \
  '--log_dir', '/content/ElasticBERT/logs/elue/entropy/STS-BTestCheck', \
  '--output_dir', '/content/ElasticBERT/ckpts/elue/entropy/STS-BTestCheck', \
  '--num_hidden_layers', '12', \
  '--num_output_layers', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', \
  '--max_seq_length', '128', \
  '--per_gpu_train_batch_size', '32', \
  '--per_gpu_eval_batch_size',' 32', \
  '--learning_rate', '2e-5', \
  '--weight_decay', '0.1', \
  '--save_steps', '50', \
  '--logging_steps', '50', \
  '--num_train_epochs', '5',  \
  '--warmup_rate', '0.06', \
  '--evaluate_during_training', \
  '--overwrite_output_dir'
]

import argparse
parser = argparse.ArgumentParser()

args = get_args(arg_vec)

In [ ]:
import torch
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend="nccl")
    args.n_gpu = 1
args.device = device

args.output_mode = 'classification'

print(args.device)
model.to(args.device)

cuda


ElasticBertForSequenceClassification(
  (elasticbert): ElasticBertModel(
    (embeddings): ElasticBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-08, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElasticBertEncoder(
      (layer): ModuleList(
        (0): ElasticBertLayer(
          (attention): ElasticBertAttention(
            (self): ElasticBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElasticBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)


In [ ]:
# !git clone https://github.com/hsm207/imdb_data.git

In [ ]:
# %cd imdb_data
# !pip install -r requirements.txt

In [ ]:
# !tf_upgrade_v2 --infile create_imdb_dataset.py --outfile bar.py

In [ ]:
# !python bar.py --output_dir /content/imdb_data/imdb_dataset

In [ ]:
#Custom Selection
dataset = 'SST-2'#'IMDb' #or 'Yelp'

#To check model performance on SST-2 dev split:
#Please set dataset = 'SST-2' and data_split='dev'

In [ ]:
import numpy as np
import pandas as pd

def get_preds(eval_dataset='SST-2', data_split='train'):
  args.spec_eval=data_split
  args.task_name=eval_dataset.lower()
  args.data_dir=ELUE_DIR + '/'+args.task_name

  results_all, exit_preds, pred_tuple, op_labels = evaluate_elue_entropy(args, model, tokenizer)


  # exit_preds_list = np.stack(exit_preds, axis=1)
  # df = pd.DataFrame((exit_preds_list) )
  # df['op_labels'] = op_labels

  return  results_all, exit_preds, pred_tuple, op_labels

In [ ]:
import logging
logger = logging.getLogger(__name__)
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
import os
from elue import elue_compute_metrics
from tqdm import tqdm
results, final_preds, pred_tuple, op_labels = get_preds(eval_dataset=dataset, data_split='train')

Evaluating: 100%|██████████| 267/267 [01:03<00:00,  4.20it/s]


In [ ]:
import numpy as np
import math
def softmax(x):
    return(np.exp(x)/np.exp(x).sum())

In [ ]:
pred_tuple[-1]

array([[ 0.07783183, -0.14257424],
       [-0.08552232, -0.02014458],
       [ 0.23074193, -0.29304874],
       ...,
       [ 0.14448684, -0.32708505],
       [ 0.05847542, -0.17466402],
       [ 0.20079498, -0.2604479 ]], dtype=float32)

In [ ]:
pred_prob_thi = []
pred_prob_las = []


for i in range(len(pred_tuple[0])):
  pred_prob_thi.append(abs((pred_tuple[1][i][0]-pred_tuple[1][i][1])))
  pred_prob_las.append(abs((pred_tuple[-1][i][0]-pred_tuple[-1][i][1])))

In [ ]:
df = pd.DataFrame(list(zip(final_preds[1], final_preds[-1], pred_prob_thi, pred_prob_las, op_labels)), columns =['Thi_layer_P', 'Last_layer', 'PProb_Thi', 'PProb_las', 'True_labels'])

In [ ]:
df1 = pd.DataFrame(list(zip(final_preds[0], final_preds[1], final_preds[2], final_preds[3], final_preds[4], final_preds[5], final_preds[6], final_preds[7], final_preds[8], final_preds[9], final_preds[10], final_preds[11], op_labels)), columns =['Fir_p', 'Sec_p', 'Thi_p', 'Fou_p', 'Fiv_p', 'Six_p', 'Sev_p', 'Eig_p', 'Nin_p', 'Ten_p', 'Ele_p', 'Twe_p', 'True_labels'])

In [ ]:
df2 = pd.DataFrame(list(zip(final_preds[0], final_preds[1], final_preds[2], final_preds[3], final_preds[4], final_preds[5], final_preds[6], op_labels)), columns =['Fir_p', 'Thi_p', 'Fou_p', 'Fiv_p', 'Sev_p', 'Nin_p', 'Twe_p', 'True_labels'])

In [ ]:
accuracy_imdb = []
for j in df1.columns:
  accuracy = 0
  for i in range(df1.shape[0]):
      if df1[j][i] == df1['True_labels'][i]:
          accuracy += 1
      else:
          pass
  print("Accuracy = ", accuracy/df1.shape[0])
  accuracy_imdb.append(accuracy)

Accuracy =  0.50584
Accuracy =  0.52
Accuracy =  0.55244
Accuracy =  0.551
Accuracy =  0.62064
Accuracy =  0.60052
Accuracy =  0.77208
Accuracy =  1.0


In [ ]:
accuracy_yelp = []
for j in df2.columns:
  accuracy = 0
  for i in range(df1.shape[0]):
      if df2[j][i] == df2['True_labels'][i]:
          accuracy += 1
      else:
          pass
  print("Accuracy = ", accuracy/df2.shape[0])
  accuracy_imdb.append(accuracy)

NameError: ignored

In [ ]:
df.to_csv("Early_Exit_Confidence_data_yelp_new_exits(3,12)_difference.csv")

In [ ]:
df1.to_csv("Early_Exit_Confidence_data_SST2_new_exits(12,12)_difference.csv")

In [ ]:
df.to_csv(r'/content/drive/MyDrive/elue_data/Exit_Predictions_TrainTest_Yelp_4exits.csv')

In [ ]:
df.shape

(8544, 5)

In [ ]:
from google.colab import files

In [ ]:
# files.download('/content/MutiExitDNNs/ElasticBERT/finetune-dynamic/Early_Exit_Confidence_data')

In [ ]:
df_train = get_preds(eval_dataset=dataset, data_split='train')
df_test = get_preds(eval_dataset=dataset, data_split='test')

df_tot = pd.concat([df_train, df_test])
df_tot = df_tot.reset_index(drop=True)
print(df_tot.head())

df_tot.to_csv(r'/content/drive/MyDrive/Early_Exits_Divya/Model_exit_predictions/Exit_Predictions_TrainTest_IMDb_8exits.csv',sep ='\t', index = False)

Evaluating: 100%|██████████| 782/782 [03:10<00:00,  4.10it/s]

   0  1  2  3  4  5  6  7  op_labels
0  0  0  0  1  0  0  0  1          1
1  0  0  0  1  0  0  0  0          1
2  0  0  0  1  0  0  0  0          1
3  0  0  0  1  0  0  1  1          1
4  0  0  1  1  0  0  1  1          1
